# Suo

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scvi

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server

sc.settings.verbosity = 3

In [3]:
torch.cuda.is_available()

False

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [5]:
adata = ad.read_h5ad("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/dataset_complete_Suo_invae.h5ad")
obsms = [i for i in adata.obsm if i not in ['Unintegrated', 'X_pca', 'harmony']]
adata

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'inveriant_1', 'inveriant_2', 'inveriant_3', 'inveriant_4', 'inveriant_5', 'inveriant_6', 'inveriant_7', 'inveriant_8'

In [6]:
biocons = BioConservation(
    isolated_labels=False,
    nmi_ari_cluster_labels_leiden=False,
    nmi_ari_cluster_labels_kmeans=False,
    silhouette_label=False,
    clisi_knn=False
)
batchcor = BatchCorrection()

In [7]:
def get_results(self, min_max_scale: bool = True) -> pd.DataFrame:
    _LABELS = "labels"
    _BATCH = "batch"
    _X_PRE = "X_pre"
    _METRIC_TYPE = "Metric Type"
    _AGGREGATE_SCORE = "Aggregate score"
    
    df = self._results.transpose()
    df.index.name = "Embedding"
    df = df.loc[df.index != _METRIC_TYPE]
    if min_max_scale:
        # Use sklearn to min max scale
        df = pd.DataFrame(
            MinMaxScaler().fit_transform(df),
            columns=df.columns,
            index=df.index,
        )
    df = df.transpose()
    df[_METRIC_TYPE] = self._results[_METRIC_TYPE].values

    # Compute scores
    per_class_score = df.groupby(_METRIC_TYPE).mean().transpose()
    # This is the default scIB weighting from the manuscript
    # per_class_score["Total"] = 0.4 * per_class_score["Batch correction"] + 0.6 * per_class_score["Bio conservation"]
    df = pd.concat([df.transpose(), per_class_score], axis=1)
    df.loc[_METRIC_TYPE, per_class_score.columns] = _AGGREGATE_SCORE
    return df

In [8]:
print(adata, flush=True)
print("starting scvi scib", flush=True)
bm = Benchmarker(
    adata=adata,
    batch_key="concatenated_integration_covariates",
    label_key="cell_type",
    embedding_obsm_keys=obsms,
    pre_integrated_embedding_obsm_key="Unintegrated",  # equals to X_pca
    bio_conservation_metrics=biocons,
    batch_correction_metrics=batchcor,
    n_jobs=-1,
)

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'inveriant_1', 'inveriant_2', 'inveriant_3', 'inveriant_4', 'inveriant_5', 'inveriant_6', 'inveriant_7', 'inveriant_8'


starting scvi scib


In [9]:
bm.benchmark()

Computing neighbors:   0%|          | 0/8 [00:00<?, ?it/s]

Computing neighbors:  12%|█▎        | 1/8 [03:18<23:10, 198.63s/it]

Computing neighbors:  25%|██▌       | 2/8 [06:36<19:50, 198.35s/it]

Computing neighbors:  38%|███▊      | 3/8 [09:45<16:10, 194.00s/it]

Computing neighbors:  50%|█████     | 4/8 [13:02<12:59, 194.95s/it]

...

Metrics:  40%|████      | 4/10 [10:01<15:29, 154.88s/it, Batch correction: pcr_comparison]    

/home/icb/kemal.inecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scib_metrics/_pcr_comparison.py:49: UserWarning: PCR comparison score is negative, meaning variance contribution increased after integration. Setting to 0.
  warnings.warn(



Metrics:  50%|█████     | 5/10 [10:05<08:21, 100.23s/it, Batch correction: pcr_comparison]

Embeddings: 100%|██████████| 8/8 [1:23:20<00:00, 611.00s/it]

Embeddings: 100%|██████████| 8/8 [1:23:20<00:00, 625.11s/it]

In [10]:
df = get_results(bm, min_max_scale=False)

In [11]:
df

,silhouette_batch,ilisi_knn,kbet_per_label,graph_connectivity,pcr_comparison,Batch correction
Embedding,,,,,,
inveriant_1,0.840425,0.045366,0.184155,0.825926,0,0.379175
inveriant_2,0.839964,0.038026,0.192607,0.812066,0,0.376533
inveriant_3,0.84182,0.045526,0.200031,0.816598,0,0.380795
inveriant_4,0.8367,0.03869,0.1922,0.819454,0,0.377409
inveriant_5,0.838111,0.043632,0.185853,0.813065,0,0.376132
inveriant_6,0.831165,0.038995,0.191968,0.829795,0,0.378385
inveriant_7,0.840279,0.040289,0.19434,0.835844,0,0.38215
inveriant_8,0.822397,0.043648,0.258391,0.857191,0,0.396325
Metric Type,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score


In [12]:
pickle_path = os.path.join(tardis.config.io_directories["temp"], "latent", "suo_scib_invae_batch.pickle")
pickle_path

'/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/_temporary/latent/suo_scib_invae_batch.pickle'

In [13]:
df.to_pickle(pickle_path)